# PromptTemplate

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# .env 파일 로드
load_dotenv()

# langSmith에 로깅 할 프로젝트 명을 입력
logging.langsmith("02-Prompt")

LangSmith 추적을 시작합니다.
[프로젝트명]
02-Prompt


In [2]:
# LLM 객체 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

### 방법 1. from_template() 메소드를 사용하여 PromptTemplate 객체 생성
- 치환될 변수를 { 변수 }로 묶어서 템플릿을 정의

In [3]:
from langchain_core.prompts import PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
chain = prompt | llm

In [4]:
print(chain.invoke({"country": "한국"}))

content='한국의 수도는 서울입니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None} id='run-175984ee-b043-4210-86e4-babe8a377ff1-0' usage_metadata={'input_tokens': 15, 'output_tokens': 8, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


### 방법 2. PromptTemplate 객체 생성과 동시에 Prompt 생성

추가 유효성 검사를 위해 `input_variables`를 명시적으로 지정
이러한 변수는 인스턴스화 중에 템플릿 문자열에 있는 변수와 비교하여 불일치 하는 경우 예외 발생

In [12]:
# template 정의
template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate(template=template, input_variables=["country"])
prompt

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')


방법 2보다는 방법 1이 더 편한 듯

### partial variables

In [17]:
template = "{country1} 과 {country2}의 수도는 각각 어디야?"
prompt = PromptTemplate.from_template(template)

prompt = prompt.partial(country1="캐나다")

chain = prompt | llm

chain.invoke({"country2": "미국"})

AIMessage(content='캐나다의 수도는 오타와이고, 미국의 수도는 워싱턴 D.C.입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 18, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac22d9e3-83dd-409f-af37-1cffe5016433-0', usage_metadata={'input_tokens': 18, 'output_tokens': 23, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

partial을 이용하여 변수에 들어갈 기본 값을 정의 해줄수 있다. 혹은 부분 변수 채움이라 표현

아래와 같이 기본 값이 정의 된 상태에서 invoke 시 새로운 변수 값을 할당해주면 Override 된다

In [18]:
chain.invoke({"country1": "네덜란드", "country2": "미국"})

AIMessage(content='네덜란드의 수도는 암스테르담이고, 미국의 수도는 워싱턴 D.C.입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 21, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-ff69435d-7eb3-42fa-8873-d16cd95e3241-0', usage_metadata={'input_tokens': 21, 'output_tokens': 28, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

partial 을 좀 더 잘 활용해보고자 한다면 아래의 예처럼 오늘의 날짜를 미리 정의하는 방법이 있을 것이다

In [19]:
from datetime import datetime


# 오늘 날짜를 반환하는 함수 정의
def get_today():
    return datetime.now().strftime("%B %d")

In [21]:
template = "오늘의 날짜는 {today} 입니다. 오늘이 생일 유명인 {n}명을 생년월일을 표기하여 나열해 주세요"
prompt = PromptTemplate.from_template(template)
prompt = prompt.partial(today=get_today())

chain = prompt | llm

In [24]:
print(chain.invoke(5).content)

February 3에 태어난 유명인 중 다섯 명은 다음과 같습니다:

1. 노먼 록웰 (Norman Rockwell) - 1894년 2월 3일
2. 모건 페어차일드 (Morgan Fairchild) - 1950년 2월 3일
3. 네이선 레인 (Nathan Lane) - 1956년 2월 3일
4. 이사벨라 스콜루프코 (Izabella Scorupco) - 1970년 2월 3일
5. 워윅 데이비스 (Warwick Davis) - 1970년 2월 3일

이들은 다양한 분야에서 활동한 유명인들입니다.


### 파일로부터 template 읽어오기

prompt를 작성하다 보면 짧은 것만 사용하는 것이 아니라 굉장히 긴 구조의 prompt를 생성하게 된다
기존 처럼 template 변수에 정의 할때 긴 내용의 prompt를 직접 넣는 것이 아니라 yaml 파일에 정의해두면 간편하게 파일만 로드해서 사용할 수 있다

In [33]:
from langchain_core.prompts import load_prompt

# load_prompt 메서드를 이용하여 yaml 파일을 로드
prompt = load_prompt("prompts/capital.yaml")
prompt

chain = prompt | llm

In [ ]:
print(chain.invoke({"country": "미국"}).content)

미국의 수도는 워싱턴 D.C.입니다.

1. 면적: 워싱턴 D.C.의 총 면적은 약 177제곱킬로미터입니다. 이는 상대적으로 작은 크기지만, 중요한 정치적 중심지로서 기능하고 있습니다.

2. 인구: 2020년 기준으로 약 70만 명이 거주하고 있습니다. 다양한 배경을 가진 사람들이 모여 있으며, 정치와 행정 관련 인구가 많습니다.

3. 역사적 장소: 워싱턴 D.C.에는 백악관, 미국 의회 의사당, 링컨 기념관, 그리고 워싱턴 기념비 등 미국의 역사와 정체성을 상징하는 많은 기념비와 박물관이 있습니다. 스미소니언 박물관과 같은 문화 기관도 유명합니다.

4. 특산품: 워싱턴 D.C. 자체로는 전통적인 농산물 특산품은 없지만, 정치적 중심지로서 정치 관련 기념품과 출판물, 그리고 다양한 박물관에서 제공하는 문화상품이 특색 있는 상품으로 꼽힙니다.


### ChatPromptTemplate

`ChatPromptTemplate`은 대화목록을 프롬프트로 주입하고자 할 때 사용할 수 있다.
메시지는 Tuple 형식으로 구성하며 (`role`, `message`)로 구성하여 리스트로 생성할 수 있다

**role**
- `"system"`: 시스템 설정 메시지로 전역설정과 관련 된 프롬프트이다
- `"human"`: 사용자 입력 메시지 프롬프트이다.
- `"ai"` : AI의 답변 메시지 프롬프트이다.

In [43]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate(
    [
        # role, message
        ("system", "당신은 친절한 AI 어시스턴트입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "안녕하세요! 무엇을 도와드릴까요?"),
        ("human", "{user_input}"),
    ]
)

In [44]:
chain = chat_prompt | llm

# 이렇게 invoke 시 직접 호출하는 방법도 있고
print(chain.invoke({"name": "춘식", "user_input": "너는 춘식이에 대해 알아?"}).content)

네, 춘식이는 카카오의 인기 캐릭터 중 하나입니다. 라이언의 친구로 등장하며, 귀여운 외모와 독특한 매력으로 많은 사랑을 받고 있습니다. 춘식이는 주로 재미있고 엉뚱한 행동을 하는 것으로 유명합니다. 더 궁금한 점이 있으면 말씀해 주세요!


In [42]:
# 이렇게 message를 생성하여 invoke 하여 직접 LLM 객체에 invoke 하는 방법도 있다
message = chat_prompt.format_messages(name="춘식", user_input="춘식이에 대해 알아?")

print(llm.invoke(message).content)

네, 춘식이는 카카오톡의 인기 캐릭터 중 하나입니다. 카카오프렌즈에 속한 캐릭터로, 귀여운 외모와 독특한 매력으로 많은 사람들에게 사랑받고 있어요. 일반적으로 춘식이는 곰으로 묘사되며, 종종 라이언과 함께 등장하기도 합니다. 더 궁금한 점 있으신가요?


### MessagePlaceholder

LangChain은 렌더링할 메시지를 완전히 제어할 수 있는 `MessagePlaceholder`를 제공

메시지 프롬프트 템플릿에 어떤 역할을 사용해야 할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용하게 사용

In [45]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약해줘."),
    ]
)

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [56]:
formatted_chat_prompt = chat_prompt.format(
    word_count=20,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 춘식입니다. 만나서 반값습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다"),
    ],
)
formatted_chat_prompt

'System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.\nHuman: 안녕하세요! 저는 오늘 새로 입사한 춘식입니다. 만나서 반값습니다.\nAI: 반가워요! 앞으로 잘 부탁 드립니다\nHuman: 지금까지의 대화를 20 단어로 요약해줘.'

In [49]:
# chain 생성
chain = chat_prompt | llm | StrOutputParser()

In [55]:
chain.invoke(
    {
        "word_count": 10,
        "conversation": [
        ("human", "안녕하세요! 저는 오늘 새로 입사한 춘식입니다. 만나서 반값습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다"),
        ]
    }
)

'새로 입사한 춘식, 반가움과 인사 나누는 대화.'